In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
import tensorflow.keras.datasets as datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable = True)

In [4]:
BATCH_SIZE = 64

(train, test), metadata = tfds.load(
    "mnist",
    split = ["train", "test"],
    shuffle_files = True,
    as_supervised = True,
    with_info = True,
)

def normalize_data(image, label):
    return tf.cast(image, tf.float32)/255.0, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
train = train.map(normalize_data, num_parallel_calls =AUTOTUNE)
train = train.cache()
train = train.shuffle(metadata.splits["train"].num_examples)
train = train.batch(BATCH_SIZE)
train = train.prefetch(AUTOTUNE)

test = test.map(normalize_data, num_parallel_calls = AUTOTUNE)
test = test.cache()
# test = test.shuffle(metadata.splits["test"].num_examples)
test = test.batch(BATCH_SIZE)
test = test.prefetch(AUTOTUNE)

In [7]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size = 3):
        super(CNNBlock, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size, padding = 'same')
        self.bn = layers.BatchNormalization()
        # self.relu = keras.activations.relu
        self.relu = layers.ReLU()
        
    def call(self, input_tensor, training = False):
        x = self.conv1(input_tensor)
        x = self.bn(x, training = training)
        x = self.relu(x)
        return x

In [20]:
old_model = keras.models.load_model("pretrained/BasicCNN/", custom_objects = {'CNNBlock': CNNBlock})
old_model.trainable = False
print("Old Model")
print(old_model.summary())
base_input = old_model.input
base_output = old_model.layers[-2].output
classification = layers.Dense(10)(base_output)
model = Model(inputs = [base_input, ], outputs = [classification, ])
print()
print("New Model")
print(model.summary())

Old Model
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 cnn_block_1 (CNNBlock)      (None, 28, 28, 32)        448       
                                                                 
 cnn_block_2 (CNNBlock)      (None, 28, 28, 64)        18752     
                                                                 
 cnn_block_3 (CNNBlock)      (None, 28, 28, 128)       74368     
                                                                 
 global_average_pooling2d (G  (None, 128)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                   

In [22]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy'],
)

In [23]:
model.fit(train, verbose = 2, epochs = 5)
print()
model.evaluate(test, verbose = 2)

Epoch 1/5
938/938 - 12s - loss: 0.4108 - accuracy: 0.9193 - 12s/epoch - 13ms/step
Epoch 2/5
938/938 - 7s - loss: 0.1149 - accuracy: 0.9717 - 7s/epoch - 8ms/step
Epoch 3/5
938/938 - 7s - loss: 0.0942 - accuracy: 0.9736 - 7s/epoch - 8ms/step
Epoch 4/5
938/938 - 7s - loss: 0.0877 - accuracy: 0.9745 - 7s/epoch - 8ms/step
Epoch 5/5
938/938 - 7s - loss: 0.0817 - accuracy: 0.9756 - 7s/epoch - 7ms/step

157/157 - 1s - loss: 0.0742 - accuracy: 0.9774 - 1s/epoch - 9ms/step


[0.07421784102916718, 0.977400004863739]

Text Dataset

In [28]:
(text_train, text_test), metadata = tfds.load(
    "imdb_reviews",
    split = ["train", "test"],
    shuffle_files = True,
    as_supervised = True,
    with_info = True,
)

tokenizer = tfds.deprecated.text.Tokenizer()
def build_vocabulary(train):
    vocab = set()
    for text, _ in train:
        vocab.update(tokenizer.tokenize(text.numpy().lower()))
    return vocab

vocab = build_vocabulary(text_train)
encoder = tfds.deprecated.text.TokenTextEncoder(
    vocab, oov_token = "<UNK>", lowercase = True, tokenizer = tokenizer
)

def encode_data(text, label):
    return encoder.encode(text.numpy()), label

def encode_map(text, label):
    encoded_text, label = tf.py_function(
        encode_data, inp = [text, label], Tout = [tf.int64, tf.int64]
    )
    encoded_text.set_shape([None])
    label.set_shape([]) 
    return encoded_text, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
text_train = text_train.map(encode_map, num_parallel_calls = AUTOTUNE).cache()
text_train = text_train.shuffle(metadata.splits["train"].num_examples)
text_train = text_train.padded_batch(32, padded_shapes=([None], ()))
text_train = text_train.prefetch(AUTOTUNE)

text_test = text_test.map(encode_map, num_parallel_calls = AUTOTUNE).cache()
text_test = text_test.padded_batch(32, padded_shapes=([None], ()))
text_test = text_test.prefetch(AUTOTUNE)

In [29]:
model_text = Sequential([
    layers.Masking(mask_value = 0),
    layers.Embedding(input_dim = len(vocab)+2, output_dim=32),
    layers.GlobalAveragePooling1D(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(1)
])

In [33]:
model_text.compile(
    loss = keras.losses.BinaryCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy'],
)

model_text.fit(text_train, verbose = 2, epochs = 5)
print()
model_text.evaluate(text_test, verbose = 2)

Epoch 1/5
782/782 - 31s - loss: 0.5344 - accuracy: 0.6736 - 31s/epoch - 40ms/step
Epoch 2/5
782/782 - 5s - loss: 0.2628 - accuracy: 0.8931 - 5s/epoch - 7ms/step
Epoch 3/5
782/782 - 6s - loss: 0.1906 - accuracy: 0.9270 - 6s/epoch - 7ms/step
Epoch 4/5
782/782 - 5s - loss: 0.1400 - accuracy: 0.9495 - 5s/epoch - 7ms/step
Epoch 5/5
782/782 - 5s - loss: 0.1074 - accuracy: 0.9642 - 5s/epoch - 7ms/step

782/782 - 32s - loss: 0.3148 - accuracy: 0.8825 - 32s/epoch - 41ms/step


[0.3148319125175476, 0.8825200200080872]

In [34]:
print(model_text.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, None)              0         
                                                                 
 embedding (Embedding)       (None, None, 32)          2396640   
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_8 (Dense)             (None, 64)                2112      
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,398,817
Trainable params: 2,398,817
Non-trainable params: 0
______________________________________________